In [1]:
import torch
import torch.nn as nn
import torch.optim as opt
import numpy as np

# CUDA 하드웨어 가속기 이용법
CUDA = NVIDIA에서 독점중인 하드웨어 가속기

In [9]:
print(torch.cuda.is_available())
#토치에서 쿠다 사용가능한지?
print(torch.cuda.current_device())
#연결된 쿠다 장치의 정체
print(torch.cuda.device_count())
#그래픽카드의 장 수
print(torch.cuda.get_device_name(0))
#그래픽카드 장치의 이름
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#앞으로 토치안의 메모리변수를 어느장치에 할당 정의

True
0
1
Tesla K80


# 모델 저장 및 불러오기

In [6]:
class model(nn.Module):
  def __init__(self):
    super(model,self).__init__()
    self.lin1 = nn.Linear(1,1)

  def forward(self,x):
    x = self.lin1(x)
    return x

In [7]:
test_model = model().to(device) #test_model은 CUDA장치에 할당
optimizer = opt.SGD(test_model.parameters(),lr=0.1)

In [8]:
x_data = np.linspace(-1,1,100)
y_data = np.sin(x_data)+np.random.uniform(-1,1,size=100)
x = torch.Tensor(x_data).view(-1,1).to(device)
y = torch.Tensor(y_data).view(-1,1).to(device) #x,y도 같은 device(CUDA)로 할당해서 계산해야함

In [10]:
print(x)

tensor([[-1.0000],
        [-0.9798],
        [-0.9596],
        [-0.9394],
        [-0.9192],
        [-0.8990],
        [-0.8788],
        [-0.8586],
        [-0.8384],
        [-0.8182],
        [-0.7980],
        [-0.7778],
        [-0.7576],
        [-0.7374],
        [-0.7172],
        [-0.6970],
        [-0.6768],
        [-0.6566],
        [-0.6364],
        [-0.6162],
        [-0.5960],
        [-0.5758],
        [-0.5556],
        [-0.5354],
        [-0.5152],
        [-0.4949],
        [-0.4747],
        [-0.4545],
        [-0.4343],
        [-0.4141],
        [-0.3939],
        [-0.3737],
        [-0.3535],
        [-0.3333],
        [-0.3131],
        [-0.2929],
        [-0.2727],
        [-0.2525],
        [-0.2323],
        [-0.2121],
        [-0.1919],
        [-0.1717],
        [-0.1515],
        [-0.1313],
        [-0.1111],
        [-0.0909],
        [-0.0707],
        [-0.0505],
        [-0.0303],
        [-0.0101],
        [ 0.0101],
        [ 0.0303],
        [ 0.

In [11]:
#test_model 학습
for ep in range(10):
  optimizer.zero_grad()
  y_infer = test_model(x)
  loss = torch.mean((y_infer-y)**2)
  loss.backward()
  optimizer.step()
  print(loss.item())

0.5914525985717773
0.5007878541946411
0.44270795583724976
0.405489444732666
0.38162848353385925
0.3663216233253479
0.356494277715683
0.3501777648925781
0.34611183404922485
0.34348922967910767


In [12]:
# 현재까지 학습이 다 완료된 상태 세이브
torch.save(test_model,'./model_save.pth')
del test_model

In [14]:
#모델 로딩!! 주의 저장된 모델에 대한 소스코드의 정보는 앞서서 정의가 되어있어야 합니다!
test_model = torch.load('./model_save.pth')

In [15]:
print(test_model)

model(
  (lin1): Linear(in_features=1, out_features=1, bias=True)
)


# Tensor --> numpy

In [16]:
x = torch.Tensor(torch.linspace(-1,1)).view(-1,1).to(device)

y_infer = test_model(x)
print(y_infer.mean())

tensor(0.0637, device='cuda:0', grad_fn=<MeanBackward0>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Not providing a value for linspace's steps is deprecated and will throw a runtime error in a future release. This warning will appear only once per process. (Triggered internally at  ../aten/src/ATen/native/RangeFactories.cpp:23.)
  """Entry point for launching an IPython kernel.


In [17]:
y_numpy = y_infer.detach().cpu().numpy()
print(y_numpy.mean())

0.063745424
